In [2]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

import numpy
import pandas
import custom
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import display


In [3]:
##데이터 불러오기
df = pandas.read_csv("data/simple_movie_data.csv", encoding="UTF8")
display(df)

with open("data/vector.pkl", mode="rb") as f:
    vec = pickle.load(f)

print(len(vec))

,quote,scoreSentiment,word_len
0,if you have not seen it do,1,7
1,it s benchmark cinema a highpoint of movie his...,1,9
2,this movie is all about brando,1,6
3,this is sean connery at his best,1,7
4,there are so many questions that the script ne...,0,10
...,...,...,...
101047,technically a little bit crude but it delivers,1,8
101048,a sturdy N entertaining late night watch N,1,8
101049,what on earth happened here N,0,6
101050,is it tuesday yet,0,4


123281


In [4]:
##데이터 전처리

sentences = df["quote"].values.tolist()
t = df["scoreSentiment"].values.tolist()
word_len = df["word_len"].max()

x = []
for s in sentences :
    temp = []
    words = custom.text_preprocess(s).split()
    for i in range(word_len - len(words)) :
        temp.append(vec["<pad>"])
    for i in range(len(words)) :
        if words[i] not in vec :
            temp.append(vec["<unk>"])
            continue
        temp.append(vec[words[i]])
    x.append(temp)
print(len(x))
    

101052


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##DataLoader 생성
tensor_x = torch.tensor(x, dtype = torch.float, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)
zip_list = list(zip(tensor_x, tensor_t))
dataloader = DataLoader(zip_list, batch_size= 200, shuffle= True)
test_dataloader = DataLoader(zip_list, batch_size= 10000, shuffle= False)

print(len(dataloader))
print(len(test_dataloader))

506
11


In [9]:
##AI 만들기
#함수 선언
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.RNN(300,10,batch_first = True,device = device)
        self.f = nn.Sequential(
            nn.Linear(10,2),
            nn.ReLU()
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x
F = NN()


In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 0.25)
epoch = 30

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
    # x, t 입력
    # y = F(x)
        y = F(x)
    # 손실함수
        loss = loss_function(y,t)
        loss_sum += loss
    # 최적화함수
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} || loss {loss_sum / len(dataloader)}")

epoch 1 || loss 0.5735529065132141
epoch 2 || loss 0.49822232127189636
epoch 3 || loss 0.47620806097984314
epoch 4 || loss 0.46683239936828613
epoch 5 || loss 0.45597314834594727
epoch 6 || loss 0.44998207688331604
epoch 7 || loss 0.4434441030025482
epoch 8 || loss 0.44048601388931274
epoch 9 || loss 0.4351338744163513
epoch 10 || loss 0.43212205171585083
epoch 11 || loss 0.43013811111450195
epoch 12 || loss 0.42872878909111023
epoch 13 || loss 0.4260614812374115
epoch 14 || loss 0.42215922474861145
epoch 15 || loss 0.4191812574863434
epoch 16 || loss 0.41903194785118103
epoch 17 || loss 0.41794073581695557
epoch 18 || loss 0.4140990078449249
epoch 19 || loss 0.4129354655742645
epoch 20 || loss 0.412267804145813
epoch 21 || loss 0.4104810357093811
epoch 22 || loss 0.41065919399261475
epoch 23 || loss 0.407453328371048
epoch 24 || loss 0.406678169965744
epoch 25 || loss 0.40706536173820496
epoch 26 || loss 0.40629100799560547
epoch 27 || loss 0.40323424339294434
epoch 28 || loss 0.40389

In [11]:
cnt = 0
total = 0

for x, t in test_dataloader :
    y = F(x)
    cnt += (y.argmax(dim=1) == t).sum().item()
    total += len(x)

print("possiblity : %f" %(cnt / total))

possiblity : 0.788376


In [12]:
torch.save(F.to("cpu"), "simple_movie_RNN.pt")